In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle
from collections import deque
import time
import pickle
import os
from datetime import datetime
from copy import deepcopy

# Problem Features

## 1- Stages of the disease

  - susceptible
  - exposed   ???????
  - presymptomatic 
  - infected asymptomatic
  - infected symptomatic
  - detected
  - recovered, with immunity
  - dead

## 2- Parameters

  - Moving probability (moving_probability) (not necessarily the same for all individuals)
  - Fraction of Symptomatic Cases (fraction_of_symptomatic)
  - Fraction of individuals equipped
with an application (contact_tracing_fraction)
  - Probability of detecting symptomatic individuals (symptomatic_detection_probability)
  - Probability of recovery after detection (recovery_after_detection_probability)
  - Probability of recovery without detection (recovery_without_detection_probability)
  - Probability of becomming contagious (becoming_contagious_probability)
  - Fatality after detection (fatality_after_detection)
  - Fatality before detection (fatality_before_detection)

## 3- Control methods

  - Contact tracing
  - Lockdown
  - Body temperature checking (From Giovanni's paper)


## 4- Goals

  - 



# Coordinate Class

Used for storing the coordinate of individuals in the lattice

In [ ]:
class Coordinate:
    
    MAX_LATTICE_SIZE = 1_000
    PRECISION = 0.0001
    
    def __init__(self, x, y, lattice_size=None):
        
        self.lattice_size = lattice_size
        self.xy = np.array([x, y])
        if (self.lattice_size!=None):
            self.xy = self.xy % self.lattice_size
        

    def __add__(self, other):
          
        xy = None
        if (type(other)==Displacement):
            xy = self.xy + other.xy
        elif (type(other)==int) or (type(other)==float):
            xy = self.xy + other
        else:
            assert (0), f"invalied + operation between {self} and {other}"
        
        if (self.lattice_size!=None):
            xy = xy % self.lattice_size
        return type(self)(*xy, self.lattice_size)
        
    
    def __mul__(self, other):
        
        assert (type(other)==float or type(other)==int), f"invalied * operation between {self} and {other}"
        xy = self.xy * other
        if (self.lattice_size!=None):
            xy = xy % self.lattice_size
        return type(self)(*xy, self.lattice_size)
    
    
    def __rmul__(self, other):
        
        return self.__mul__(other)
    
    
    def __eq__(self, other):
        
        return (self.xy==other.xy).all() and (self.lattice_size==other.lattice_size)
    
    
    def __repr__(self):
        
        return f"Coordinate({self.xy[0]}, {self.xy[1]})"
    
    
    def __str__(self):
        
        return self.__repr__()
    
    
    def __hash__(self):
        
        return int((self.xy[0] * self.MAX_LATTICE_SIZE + self.xy[1])/self.PRECISION)
    
    
    @property
    def tolist(self):

        return self.xy.tolist()

# Displacement Class

Used for defining the direction of individuals' movements

In [ ]:
class Displacement(Coordinate):
    

    def __init__(self, x, y, lattice_size=None):

        super().__init__(x, y, lattice_size)

    
    def __repr__(self):
        
        return f"Displacement({self.xy[0]}, {self.xy[1]})"
    
    
    def __str__(self):
        
        return self.__repr__()

# Generate Random Coordinates

Used for generating new coordinates for initializing individuals in the lattice

In [ ]:
def get_random_coordinate(lattice_size):
    
    assert (type(lattice_size)==int)
    
    xy = np.random.randint(lattice_size, size=(2,))
    
    return Coordinate(*xy, lattice_size)

# Default Parameters of Problem

In [ ]:
TIME_CONSTANT = 5                                       # Number of time steps that make on day

N_INDIVIDUALS_DEFAULT = 250                             # Chosen
LATTICE_SIZE_DEFAULT = 50                               # Chosen
INITIAL_INFECTION_RATE_DEFAULT = 0.01                   # Chosen

MOVING_RATE_RANGE_DEFAULT = [0.6, 0.8]                  # Chosen
PRESYMPTOMATIC_STAGE_MOVEMENT_FREEDOM = 1.              # Chosen
DETECTED_STAGE_MOVEMENT_FREEDOM = 1.                    # Chosen

INFECTION_PROBABILITY_DEFAULT = 0.5                     # Chosen
SYMPTOMATIC_INFECTION_PROBABILITY_DEFAULT = 0.85        # From the paper

AVERAGE_DAYS_OF_IMMUNITY = 75.                          # Coording to CDC, after recovery, each individual has at leaset 2-3 months of immunity
AVERAGE_DAYS_UNTIL_DITECTING_SYMPTOMATIC_INDIVIDUAL = 3.# Chosen
AVERAGE_DAYS_UNTIL_BECOMING_PRESYMPATIC = 4.            # From the paper
AVERAGE_DAYS_UNTIL_BECOMING_CONTAGIOUS = 4.             # From the paper
AVERAGE_DAYS_UNTIL_RECOVERY_AFTER_DETECTION = 4.        # Chosen
AVERAGE_DAYS_UNTIL_RECOVERY_WITHOUT_DETECTION = 15.     # From the paper

IMMUNITY_PROBABILITY = 1#-1./(AVERAGE_DAYS_OF_IMMUNITY*TIME_CONSTANT)                         
SYMPTOMATIC_DETECTION_PROBABILITY_DEFAULT = 1./(AVERAGE_DAYS_UNTIL_DITECTING_SYMPTOMATIC_INDIVIDUAL*TIME_CONSTANT)
BECOMING_PRESYMPATIC_ROBABILITY_DEFAULT = 1./(AVERAGE_DAYS_UNTIL_BECOMING_PRESYMPATIC*TIME_CONSTANT)
BECOMING_CONTAGIOUS_PROBABILITY_DEFAULT = 1./(AVERAGE_DAYS_UNTIL_BECOMING_CONTAGIOUS*TIME_CONSTANT)
RECOVERY_AFTER_DETECTION_PROBABILITY_DEFAULT = 1./(AVERAGE_DAYS_UNTIL_RECOVERY_AFTER_DETECTION*TIME_CONSTANT)
RECOVERY_WITHOUT_DETECTION_PROBABILITY_DEFAULT = 1./(AVERAGE_DAYS_UNTIL_RECOVERY_WITHOUT_DETECTION*TIME_CONSTANT)

average_recovery_steps_in_symptomatic_infection = (SYMPTOMATIC_DETECTION_PROBABILITY_DEFAULT/RECOVERY_AFTER_DETECTION_PROBABILITY_DEFAULT+RECOVERY_WITHOUT_DETECTION_PROBABILITY_DEFAULT)/(SYMPTOMATIC_DETECTION_PROBABILITY_DEFAULT+RECOVERY_AFTER_DETECTION_PROBABILITY_DEFAULT)

AVERAGE_INFECTION_DAYS = (1/BECOMING_PRESYMPATIC_ROBABILITY_DEFAULT +\
                          1/BECOMING_CONTAGIOUS_PROBABILITY_DEFAULT +\
                         (1-SYMPTOMATIC_INFECTION_PROBABILITY_DEFAULT)*(1/RECOVERY_WITHOUT_DETECTION_PROBABILITY_DEFAULT) +\
                         SYMPTOMATIC_INFECTION_PROBABILITY_DEFAULT * average_recovery_steps_in_symptomatic_infection)/TIME_CONSTANT


TRACED_INDIVIDUALS_FRACTION_DEFAULT = 0.4               # Chosen
DAYS_TO_TRAC_INDIVIDUALS = AVERAGE_INFECTION_DAYS

TEST_ACCURACY = 0.95                                    # According to the information on available test for covid

N_TIME_STEPS_TO_TRACE_DEFAULT = int(DAYS_TO_TRAC_INDIVIDUALS*TIME_CONSTANT)


# Total covid cases and death (Dec 2nd)
# Cases: 264M             Deaths: 5,22M
# Assume: Getting detected can increase the chance of survival by a factor of three
FATALITY_AFTER_DETECTION_PROBABILITY_DEFAULT = 0       # 5.22/264.*(0.5)/TIME_CONSTANT
FATALITY_BEFORE_DETECTION_PROBABILITY_DEFAULT = 0      # 5.22/264.*(1.5)/TIME_CONSTANT

INITIALLY_TESTED_POPULATION_FRACTION = 0.00

# Individual Info Class

Used for storing the disease-related information of individuals

In [ ]:
class DiseaseInfo:
    
    def __init__(self,
                 infection_probability=INFECTION_PROBABILITY_DEFAULT,
                 becoming_presymptomatic_probability=BECOMING_PRESYMPATIC_ROBABILITY_DEFAULT,
                 becoming_contagious_probability=BECOMING_CONTAGIOUS_PROBABILITY_DEFAULT,
                 symptomatic_infection_probability=SYMPTOMATIC_INFECTION_PROBABILITY_DEFAULT,
                 symptomatic_detection_probability=SYMPTOMATIC_DETECTION_PROBABILITY_DEFAULT,
                 recovery_after_detection_probability=RECOVERY_AFTER_DETECTION_PROBABILITY_DEFAULT,
                 recovery_without_detection_probability=RECOVERY_WITHOUT_DETECTION_PROBABILITY_DEFAULT,
                 fatality_after_detection_probability=FATALITY_AFTER_DETECTION_PROBABILITY_DEFAULT,
                 fatality_before_detection_probability=FATALITY_BEFORE_DETECTION_PROBABILITY_DEFAULT,
                 immunity_probability=IMMUNITY_PROBABILITY):
        
        self.infection_probability = infection_probability
        self.becoming_presymptomatic_probability = becoming_presymptomatic_probability
        self.becoming_contagious_probability = becoming_contagious_probability
        self.symptomatic_infection_probability = symptomatic_infection_probability
        self.symptomatic_detection_probability = symptomatic_detection_probability
        self.recovery_after_detection_probability = recovery_after_detection_probability
        self.recovery_without_detection_probability = recovery_without_detection_probability
        self.fatality_after_detection_probability = fatality_after_detection_probability
        self.fatality_before_detection_probability = fatality_before_detection_probability
        self.immunity_probability = immunity_probability
        

    def __repr__(self):

        return f'Infection Probability: {self.infection_probability}\n'\
               f'Becoming Presymptomatic Probability: {self.becoming_presymptomatic_probability}\n'\
               f'Becoming Contagious Probability: {self.becoming_contagious_probability}\n'\
               f'Symptomatic Infection Probability: {self.symptomatic_infection_probability}\n'\
               f'Symptomatic Detection Probability: {self.symptomatic_detection_probability}\n'\
               f'Recovery After Detection Probability: {self.recovery_after_detection_probability}\n'\
               f'Recovery Without Detection Probability: {self.recovery_without_detection_probability}\n'\
               f'Fatality After Detection Probability: {self.fatality_after_detection_probability}\n'\
               f'Fatality Before Detection Probability: {self.fatality_before_detection_probability}\n'\
               f'Immunity Probability: {self.immunity_probability}'
        
    
    def __str__(self):
        
        return self.__repr__()

# Individual Class (Abstract Class)

Contains the basic atributes and methods of individuals regardless of the epidemic category they belong to

In [ ]:
class Individual(ABC):
    

    VALIED_MOVING_DIRECTIONS = [Displacement(1,0), 
                                Displacement(0,1), 
                                Displacement(-1,0), 
                                Displacement(0,-1)]
    
    
    def __init__(self, coordinate, moving_probability, disease_info):
        
        assert (type(coordinate)==Coordinate), "Invalied coordinate"
        assert (type(disease_info)==DiseaseInfo), "Invalied disease info"
        assert (moving_probability<=1 and moving_probability>=0), "Invalied moving probability"
        self.coordinate = coordinate
        self.disease_info = disease_info
        self.moving_probability = moving_probability
    
    
    @abstractmethod
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        pass
    
    
    def update_position(self, motion_restriction_coeff=0):

        if (np.random.uniform()<((1-motion_restriction_coeff)*self.stage_movemet_freedom*self.moving_probability)):
            self.move()
            
    
    def move(self):
        
        direction = self.select_random_direction()
        self.coordinate += direction
            
    
    def select_random_direction(self):
        
        movement_idx = np.random.randint(len(self.VALIED_MOVING_DIRECTIONS))
        return self.VALIED_MOVING_DIRECTIONS[movement_idx]
           
        
    @property
    def type(self):

        return type(self)


    @property
    def stage_movemet_freedom(self):

        return 1.


    @abstractmethod
    def test_for_disease(self):

        pass

# Dead Class (Inheriting from Individuals Class)

Used for modeling a dead individual

In [ ]:
class Dead(Individual):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        return self, False

    @property
    def stage_movemet_freedom(self):

        return 0
        

    def test_for_disease(self):

        return self

# Infected Class (Abstract Class Inheriting from Individuals Class)

Used for modeling an infected individual

In [ ]:
class Infected(Individual):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    @abstractmethod
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        pass


    def test_for_disease(self):
        
        if (np.random.uniform()<TEST_ACCURACY):
            return Detected(self.coordinate, self.moving_probability, self.disease_info)
        return self

# Healthy Class (Abstract Class Inheriting from Individuals Class)

Used for modeling a healthy individual

In [ ]:
class Healthy(Individual):
    

    def __init__(self, coordinate, moving_probability, disease_info, infected_before):
        
        super().__init__(coordinate, moving_probability, disease_info)
        self.infected_before = infected_before


    @property
    @abstractmethod
    def immunity_probability(self):
        pass


    def test_for_disease(self):
        
        if (np.random.uniform()<TEST_ACCURACY):
            return self
        return Detected(self.coordinate, self.moving_probability, self.disease_info)

# Susceptible Class (Inheriting from Healthy Class)

Used for modeling a susceptible individual

In [ ]:
class Susceptible(Healthy):
    

    def __init__(self, coordinate, moving_probability, disease_info, infected_before=False):
        
        super().__init__(coordinate, moving_probability, disease_info, infected_before)


    @property
    def immunity_probability(self):
        
        return 0

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        if (exposed_to_infection and (np.random.uniform()<self.disease_info.infection_probability) and (np.random.uniform()<1-self.immunity_probability)):
            individual = Exposed(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual, False

# Exposed Class (Inheriting from Healthy Class)

Used for modeling an exposed individual

In [ ]:
class Exposed(Infected):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        if (np.random.uniform()<self.disease_info.becoming_presymptomatic_probability):
            individual = PreSymptomatic(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual, False

# PreSymptomatic Class (Inheriting from Infected Class)

Used for modeling an infected pre-symptomatic individual

In [ ]:
class PreSymptomatic(Infected):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        if (np.random.uniform()<self.disease_info.becoming_contagious_probability):
            if (np.random.uniform()<self.disease_info.symptomatic_infection_probability):
                individual = InfectedSymptomatic(self.coordinate, self.moving_probability, self.disease_info)
            else:
                individual = InfectedAsymptomatic(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual, False

# InfectedSymptomatic Class (Inheriting from Infected Class)

Used for modeling an infected individual with symptoms

In [ ]:
class InfectedSymptomatic(Infected):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        r = np.random.uniform()
        sd = False
        if (r<self.disease_info.symptomatic_detection_probability):
            individual = Detected(self.coordinate, self.moving_probability, self.disease_info)
            sd = True
        elif (r<self.disease_info.symptomatic_detection_probability+
                self.disease_info.recovery_without_detection_probability):
            individual = Recovered(self.coordinate, self.moving_probability, self.disease_info)
        elif (r<self.disease_info.symptomatic_detection_probability+
                self.disease_info.recovery_without_detection_probability+
                self.disease_info.fatality_before_detection_probability):
            individual = Dead(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual , sd 

    
    @property
    def stage_movemet_freedom(self):

        return PRESYMPTOMATIC_STAGE_MOVEMENT_FREEDOM

# InfectedAsymptomatic Class (Inheriting from Infected Class)

Used for modeling an infected individual without symptoms

In [ ]:
class InfectedAsymptomatic(Infected):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        r = np.random.uniform()
        if (r<self.disease_info.recovery_without_detection_probability):
            individual = Recovered(self.coordinate, self.moving_probability, self.disease_info)
        elif (r<self.disease_info.recovery_without_detection_probability+
                self.disease_info.fatality_before_detection_probability):
            individual = Dead(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual, False

# Detected Class (Inheriting from Infected Class)

Used for modeling a positive detected individual

In [ ]:
class Detected(Infected):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info)

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        r = np.random.uniform()
        if (r<self.disease_info.recovery_after_detection_probability):
            individual = Recovered(self.coordinate, self.moving_probability, self.disease_info)
        elif (r<self.disease_info.recovery_after_detection_probability+
                self.disease_info.fatality_after_detection_probability):
            individual = Dead(self.coordinate, self.moving_probability, self.disease_info)
        individual.update_position(motion_restriction_coeff)
        return individual, False 


    @property
    def stage_movemet_freedom(self):

        return DETECTED_STAGE_MOVEMENT_FREEDOM

# Recovered Class (Inheriting from Healthy Class)

Used for modeling a recovered individual

In [ ]:
class Recovered(Healthy):
    

    def __init__(self, coordinate, moving_probability, disease_info):
        
        super().__init__(coordinate, moving_probability, disease_info, infected_before=True)


    @property
    def immunity_probability(self):
        
        return self.disease_info.immunity_probability

    
    def step_time(self, exposed_to_infection, motion_restriction_coeff=0):

        individual = self
        if (np.random.uniform()<1-self.immunity_probability):
            individual = Susceptible(self.coordinate, self.moving_probability, self.disease_info, infected_before=True)
        individual.update_position(motion_restriction_coeff)
        return individual, False

# Contact Tracing Class

Used for modeling contact tracing in pandemic

In [ ]:
class ContantTracing:


    def __init__(self, population, traced_individuals_indices, n_time_steps_to_trace, lattice_size):

        assert(np.min(traced_individuals_indices)>=0 and np.max(traced_individuals_indices)<len(population) and traced_individuals_indices.dtype==int), "Invalied traced_individuals_indices"
        self.population = population
        self.n_population = len(population)
        self.traced_individuals_indices = traced_individuals_indices
        self.lattice_size = lattice_size
        self.contacts = [deque(maxlen=n_time_steps_to_trace) if i in self.traced_individuals_indices else None for i in range(self.n_population)]


    def get_lattice(self):   
        
        lattice = [[[] for _ in range(self.lattice_size)] for _ in range(self.lattice_size)]
        for individual_idx in self.traced_individuals_indices:
            coordinate = self.population[individual_idx].coordinate
            lattice[coordinate.xy[0]][coordinate.xy[1]].append(individual_idx)
        return lattice


    def get_positive_detected_individuals(self):

        positive_detected_individuals = set()
        for individual_idx in self.traced_individuals_indices:
            individual_type = self.population[individual_idx].type
            if (individual_type==Detected):
                positive_detected_individuals.add(individual_idx)
        return positive_detected_individuals


    def update_contacts(self):

        lattice = self.get_lattice()
        for individual_idx in self.traced_individuals_indices:
            coordinate = self.population[individual_idx].coordinate
            self.contacts[individual_idx].append(lattice[coordinate.xy[0]][coordinate.xy[1]])
    
    
    def step_time(self):

        self.update_contacts()
        probably_infected_individuals_indices = []
        for individual_idx in self.get_positive_detected_individuals():
            for contacts_in_each_time_step in self.contacts[individual_idx]:
                probably_infected_individuals_indices.extend(contacts_in_each_time_step)
            self.contacts[individual_idx].clear()
        n_detections = self.test_individuals(set(probably_infected_individuals_indices))
        return n_detections

    
    def test_individuals(self, individuals_indices):

        n_detections = 0
        for individual_idx in individuals_indices:
            if (self.population[individual_idx].type!=Detected):
                self.population[individual_idx] = self.population[individual_idx].test_for_disease()
                if (self.population[individual_idx].type==Detected):
                    n_detections += 1
        return n_detections

    def test_random_individuals(self, n_tests):

        sellected_indices = np.random.permutation(self.traced_individuals_indices)[:n_tests]
        _ = self.test_individuals(sellected_indices)

# InfectionStatistics
Used for storing the statistices related to the spread of hte disease in socienty at each time step.

In [ ]:
class InfectionStatistics:
    
    
    def __init__(self, 
                 n_susceptible, 
                 n_exposed,
                 n_presymptomatic, 
                 n_asymptomatic, 
                 n_symptomatic, 
                 n_detected,
                 n_recovered,
                 n_dead,
                 lockdown_effectiveness,
                 n_ct_detections,
                 n_sd_detections):
        
        self.n_susceptible = n_susceptible
        self.n_exposed = n_exposed
        self.n_presymptomatic = n_presymptomatic
        self.n_asymptomatic  = n_asymptomatic
        self.n_symptomatic = n_symptomatic
        self.n_detected = n_detected
        self.n_recovered = n_recovered
        self.n_dead = n_dead
        self.lockdown_effectiveness = lockdown_effectiveness
        self.n_ct_detections = n_ct_detections
        self.n_sd_detections = n_sd_detections
        
        
    def __repr__(self):

        return f'Number of Susceptible: {self.n_susceptible}\n'\
               f'Number of Exposed: {self.n_exposed}\n'\
               f'Number of Presymptomatic: {self.n_presymptomatic}\n'\
               f'Number of Asymptomatic: {self.n_asymptomatic}\n'\
               f'Number of Symptomatic: {self.n_symptomatic}\n'\
               f'Number of Detected: {self.n_detected}\n'\
               f'Number of Recovered: {self.n_recovered}\n'\
               f'Number of Dead: {self.n_dead}\n'\
               f'Lockdown Effectiveness: {self.lockdown_effectiveness}'
        
    
    def __str__(self):
        
        return self.__repr__()

# Society Class

Used for modeling a society in pandemic

In [ ]:
class Society:
    

    def __init__(self, 
                 n_individuals=N_INDIVIDUALS_DEFAULT, 
                 lattice_size=LATTICE_SIZE_DEFAULT, 
                 initial_infection_rate=INITIAL_INFECTION_RATE_DEFAULT,
                 moving_rate_range=MOVING_RATE_RANGE_DEFAULT,
                 traced_indididuals_fraction=TRACED_INDIVIDUALS_FRACTION_DEFAULT,
                 n_time_steps_to_trace=N_TIME_STEPS_TO_TRACE_DEFAULT, 
                 initially_tested_population_fraction=INITIALLY_TESTED_POPULATION_FRACTION,
                 disease_info=DiseaseInfo(), 
                 infection_coordinates=[]): 

        assert (type(lattice_size)==int) and (lattice_size>0), "Invalied lattice_size" 
        assert (initial_infection_rate>0) and  (initial_infection_rate<1), "Invalied initial_infection_rate" 
        assert type(disease_info==DiseaseInfo), "Invalied disease_info" 
        assert type(moving_rate_range==list), "Invalied moving_rate_range" 
        assert len(moving_rate_range)==2, "Invalied moving_rate_range" 
        assert max(moving_rate_range)<=1 and min(moving_rate_range)>=0, "Invalied moving_rate_range" 
        
        self.n_individuals = n_individuals
        
        self.initial_infection_rate = initial_infection_rate

        self.disease_info = disease_info
        
        self.lattice_size = lattice_size

        self.n_susceptible = 0
        self.n_exposed = 0
        self.n_presymptomatic = 0
        self.n_asymptomatic = 0
        self.n_symptomatic = 0
        self.n_detected = 0
        self.n_recovered = 0
        self.n_dead = 0
        
        if infection_coordinates==[]:
            for _ in range(int(n_individuals*initial_infection_rate)):
                infection_coordinates.append(get_random_coordinate(self.lattice_size))
        
        self.population = []
        self.infected_spots = set()
        for coordinate in infection_coordinates:
            moving_probability = np.random.uniform(*moving_rate_range)
            self.population.append(PreSymptomatic(coordinate, moving_probability, self.disease_info))
            self.n_presymptomatic += 1
            self.infected_spots.add(coordinate)
        while (len(self.population)!=n_individuals):
            coordinate = get_random_coordinate(self.lattice_size)
            moving_probability = np.random.uniform(*moving_rate_range)
            self.population.append(Susceptible(coordinate, moving_probability, self.disease_info))
            self.n_susceptible += 1

        self.do_contact_tracing = True if (traced_indididuals_fraction>0) else False
        n_ct_ditections = 0
        if self.do_contact_tracing:
            traced_individuals_indices = np.random.permutation(self.n_individuals)[:int(traced_indididuals_fraction*self.n_individuals)]
            self.contact_tracing = ContantTracing(self.population, traced_individuals_indices, n_time_steps_to_trace=n_time_steps_to_trace, lattice_size=self.lattice_size)
            self.contact_tracing.test_random_individuals(int(n_individuals*initially_tested_population_fraction))
            # n_ct_ditections = self.contact_tracing.step_time()
        
        self.log = []
        self.log_info(lockdown_effectiveness=0, n_ct_ditections=0, n_sd_ditections=0)
        
    
    def iterate(self, n_iterations=None, lockdown_iterations=[], lockdown_effectiveness=0, print_results=True, verbose=False):
        
        if (n_iterations==None): n_iterations = np.inf
        if (print_results and verbose): print(f"Iteration {0}:\t\t{self.log[-1]}\t\t\r", end='')
        if (print_results and (not verbose)): print(f"Iteration {0}\t\t\r", end='')
        iteration = 1
        while True:
            lockdown = True if iteration in lockdown_iterations else False
            motion_restriction_coeff = lockdown_effectiveness if lockdown else 0
            self.step_time(motion_restriction_coeff)
            if (print_results and verbose): print(f"Iteration {iteration}:\t\t{self.log[-1]}\t\t\r", end='')
            if (print_results and not verbose): print(f"Iteration {iteration}\t\t\r", end='')
            if ((self.n_exposed+self.n_presymptomatic+self.n_asymptomatic+self.n_symptomatic+self.n_detected)==0 or iteration==n_iterations): 
                break
            iteration += 1
    
    
    def step_time(self, motion_restriction_coeff):
        
        n_ct_ditections = 0
        if (self.do_contact_tracing): 
            n_ct_ditections = self.contact_tracing.step_time()
        
        infected_spots = set()
        susceptible = 0
        exposed = 0
        presymptomatic = 0
        asymptomatic = 0
        symptomatic = 0
        detected = 0
        recovered = 0
        dead = 0
        n_sd_ditections = 0
        
        for idx in range(len(self.population)): 
            exposed_to_infection = True if self.population[idx].coordinate in self.infected_spots else False
            self.population[idx], sd_detected = self.population[idx].step_time(exposed_to_infection, motion_restriction_coeff)
            n_sd_ditections += int(sd_detected)
            individual_type = self.population[idx].type
            if (individual_type==Susceptible):
                susceptible += 1
            elif (individual_type==Exposed):
                exposed += 1
            elif (individual_type==PreSymptomatic):
                presymptomatic += 1
                infected_spots.add(self.population[idx].coordinate)
            elif (individual_type==InfectedAsymptomatic):
                asymptomatic += 1
                infected_spots.add(self.population[idx].coordinate)
            elif (individual_type==InfectedSymptomatic):
                symptomatic += 1
                infected_spots.add(self.population[idx].coordinate)
            elif (individual_type==Detected):
                detected += 1
                infected_spots.add(self.population[idx].coordinate)
            elif (individual_type==Recovered):
                recovered += 1
            elif (individual_type==Dead):
                dead += 1
        
        self.infected_spots = infected_spots
        self.n_susceptible = susceptible
        self.n_exposed = exposed
        self.n_presymptomatic = presymptomatic
        self.n_asymptomatic  = asymptomatic
        self.n_symptomatic = symptomatic
        self.n_detected = detected
        self.n_recovered = recovered
        self.n_dead = dead
        
        self.log_info(motion_restriction_coeff, n_ct_ditections, n_sd_ditections)
        
            
    
    def log_info(self, lockdown_effectiveness, n_ct_ditections, n_sd_ditections):

        self.log.append(InfectionStatistics(self.n_susceptible, 
                                            self.n_exposed,
                                            self.n_presymptomatic, 
                                            self.n_asymptomatic, 
                                            self.n_symptomatic, 
                                            self.n_detected,
                                            self.n_recovered,
                                            self.n_dead,
                                            lockdown_effectiveness, 
                                            n_ct_ditections,
                                            n_sd_ditections))

In [ ]:
n_sd_probabilities = n_ct_probabilities = 20
sd_probabilities = np.array(list(range(0,n_sd_probabilities+1)))/(n_sd_probabilities)/2
ct_probabilities = np.array(list(range(0,n_ct_probabilities+1)))/(n_ct_probabilities)
N = 20

run_log = [[[] for _ in ct_probabilities] for _ in sd_probabilities]

infection_coordinates = [get_random_coordinate(LATTICE_SIZE_DEFAULT) for _ in range(int(N_INDIVIDUALS_DEFAULT*INITIAL_INFECTION_RATE_DEFAULT))]

counter = 1
for n in range(N):
    for idx1, sd in enumerate(sd_probabilities):
        for idx2, ct in enumerate(ct_probabilities):
            print(f"run {counter}/{n_sd_probabilities*n_ct_probabilities*N}\t\tSD: {sd}\tCT: {ct}")
            disease_info = DiseaseInfo(symptomatic_detection_probability=sd)
            society = Society(traced_indididuals_fraction=ct, infection_coordinates=deepcopy(infection_coordinates), disease_info=disease_info)
            society.iterate(n_iterations=100000, print_results=True, verbose=False)
            run_log[idx1][idx2].append(society.log)
            counter += 1
    print("\n", np.array([[np.mean([n[-1].n_recovered for n in y]) for y in x] for x in run_log]))